In [1]:
import json
import os
from pprint import pprint

import numpy as np
import pandas as pd
import scipy.stats as stats

In [16]:
def find_closest_object(ext_obj, sim_objs):
        ext_xpos = ext_obj["attributes"]['xpos']
        ext_ypos = ext_obj["attributes"]['ypos']
        ext_pos = np.array([ext_xpos, ext_ypos])
        sim_pos = np.array([[obj["attributes"]['xpos'], obj["attributes"]['ypos']] for obj in sim_objs])
        dists = np.linalg.norm(sim_pos - ext_pos, axis=1)
        closest_idx = np.argmin(dists)
        return sim_objs[closest_idx]

def transform_data(scene_sim, scene_ext):
    sim_objects = scene_sim['objects']
    ext_objects = scene_ext['objects']

    for obj in ext_objects:
        closest_obj = find_closest_object(obj, sim_objects)
        obj["description"] = closest_obj["description"]
    
    scene_ext["objects"] = ext_objects
    return scene_ext

In [19]:
base_dir_sim = os.path.join(os.getenv('HOME'), "Corpora/cogentb-simulated/scenes/val") # to change!!!
base_dir_ext = os.path.join(os.getenv('HOME'), "Corpora/cogentb-extracted/scenes/val") # to change!!!


for root, dirs, files in os.walk(base_dir_sim):
    for fname in files:
        with open(os.path.join(root, fname)) as f:
            scene_sim = json.load(f)

        with open(os.path.join(base_dir_ext, fname)) as f:
            scene_ext = json.load(f)
            scene_ext = transform_data(scene_sim, scene_ext)

        with open(os.path.join(base_dir_ext, fname), "w") as f:
            json.dump(scene_ext, f)
        